In [18]:
import os
import matplotlib
from PIL import Image

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [2]:
class_lookup = {}
id_lookup = {}
with open("./tiny-imagenet/tiny-imagenet-200/wnids.txt", "r") as f:
    for i, class_id in enumerate(f):
        class_id = class_id.strip()
        class_lookup[i] = class_id
        id_lookup[class_id] = i

In [3]:
name_lookup = {}
with open("./tiny-imagenet/tiny-imagenet-200/words.txt", "r") as f:
    for line in f:
        _class, name = line.strip().split("\t")
        name_lookup[_class] = name

In [19]:
def load_training():
    prefix = "./tiny-imagenet/tiny-imagenet-200/train"
    files = os.listdir(prefix)
    x_train = []
    y_train = []
    for folder in files:
        _id = id_lookup[folder]
        images = os.listdir("%s/%s/images" % (prefix, folder))
        for image in images:
            img = Image.open("%s/%s/images/%s" % (prefix, folder, image))
            #img = img.resize((224,224), Image.ANTIALIAS)
            im = np.array(img)
            if len(im.shape) == 2:
                im = np.stack((im,)*3, axis=-1)
            x_train.append(im)
            y_train.append(int(_id))
    return np.array(x_train), np.array(y_train)

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [10]:
y_label_cnn = to_categorical(y_label, num_classes=200)
x_train_cnn = x_train / 255

In [14]:
model = keras.Sequential([
    Conv2D(64, 3, padding='same', activation='relu', input_shape=(224, 224, 3)),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPool2D((2,2), strides=2),
    Conv2D(128, 3, padding='same', activation='relu'),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPool2D((2,2), strides=2),
    Conv2D(256, 3, padding='same', activation='relu'),
    Conv2D(256, 3, padding='same', activation='relu'),
    Conv2D(256, 3, padding='same', activation='relu'),
    MaxPool2D((2,2), strides=2),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    MaxPool2D((2,2), strides=2),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(200, activation='softmax')
])

In [15]:
model.compile(
    optimizer=tf.train.AdamOptimizer(),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [16]:
model.fit(x_train_cnn, y_label_cnn, epochs=10)

Epoch 1/10
100000/100000 [==============================] - 71s 711us/step - loss: 5.2996 - acc: 0.0046
Epoch 2/10
100000/100000 [==============================] - 70s 699us/step - loss: 5.2994 - acc: 0.0044
Epoch 3/10
100000/100000 [==============================] - 70s 700us/step - loss: 5.2994 - acc: 0.0045
Epoch 4/10
100000/100000 [==============================] - 70s 699us/step - loss: 5.2994 - acc: 0.0043
Epoch 5/10
100000/100000 [==============================] - 70s 696us/step - loss: 5.2994 - acc: 0.00482s - ETA: 0s - loss: 5.2994 - acc:
Epoch 6/10
 96736/100000 [============================>.] - ETA: 2s - loss: 5.2994 - acc: 0.0044

KeyboardInterrupt: 

In [ ]:
im.shape